# Solving Sudoku Puzzles
## Assignment Preamble
Please ensure you carefully read all of the details and instructions on the assignment page, this section, and the rest of the notebook. If anything is unclear at any time please post on the forum or ask a tutor well in advance of the assignment deadline.

In addition to all of the instructions in the body of the assignment below, you must also follow the following technical instructions for all assignments in this unit. *Failure to do so may result in a grade of zero.*
* [At the bottom of the page](#Submission-Test) is some code which checks you meet the submission requirements. You **must** ensure that this runs correctly before submission.
* Do not modify or delete any of the cells that are marked as test cells, even if they appear to be empty.
* Do not duplicate any cells in the notebook – this can break the marking script. Instead, insert a new cell (e.g. from the menu) and copy across any contents as necessary.

Remember to save and backup your work regularly, and double-check you are submitting the correct version.

This notebook is the primary reference for your submission. You may write code in separate `.py` files but it must be clearly imported into the notebook so that it runs without needing to reference those files, and you must explain clearly what functionality is contained in those files (through comments, markdown cells, etc).

As always, **the work you submit for this assignment must be entirely your own.** Do not copy or work with other students. Do not copy answers that you find online. These assignments are designed to help improve your understanding first and foremost – the process of doing the assignment is part of *learning*. They are also used to assess your ability, and so you must uphold academic integrity. Submitting plagiarised work risks your entire place on your degree.

**You must implement your own work.** You can and are encouraged to use helper libraries like `numpy`, but you must not use 3rd-party libraries or code that solves the assignment for you. *Even if it is properly referenced*, you only get marks for work you have done. If you are unsure, ask on the forum or ask a tutor directly.

**The pass mark for this assignment is 40%.** We expect that students, on average, will be able to produce a submission which gets a mark between 50-70% within the normal workload allocation for the unit, but this will vary depending on individual backgrounds. Please ask for help if you are struggling.

## Getting Started
For this assignment, you will be writing an agent that can solve sudoku puzzles. You should be familiar with sudoku puzzles from the unit material. You are given a 9x9 grid with some fixed values. To solve the puzzle, the objective is to fill the empty cells of the grid such that the numbers 1 to 9 appear exactly once in each row, column, and 3x3 block of the grid. 

Below is a sample puzzle along with its solution. 

<img src="images/sudoku.png" style="width: 50%;"/>

For this assignment you will need to submit:
1. The implementation for an agent which can solve sudoku puzzles – this notebook
 * You can use any algorithm you like, from the unit material or otherwise
 * Your code will be subject to automated testing, from which grades will be assigned based on whether it can solve sudokus of varying difficulty
 * To get a high grade on this assignment, the speed of your code will also be a factor – the quicker the better
 * There are some sample tests included below, make sure your code is compatible with the format of these tests
2. A text file that explains your approach and the decisions you made in your own words – a readme file
 * Submissions that do not include the written section will receive zero marks – **this part is mandatory**
 * You may write your file in plain text (.txt) or [Markdown](https://www.markdownguide.org/basic-syntax/) (.md)
 * To get top marks on this assignment, as well as getting a high grade from your implementation, you must also demonstrate excellent academic presentation in your written section

### Choice of Algorithm
The choice of algorithm to solve sudoku puzzles is up to you. We expect you will use search techniques from the unit, but you could make something up yourself, or do some independent research to find something else. You will need to evaluate and balance the trade-off between how well suited you think the algorithm is and how difficult it is to write, but there is some advice below.

You should also consider what *data structures* you use within your code. Certain algorithms will work much faster with certain data structures, and understanding this is considered part of understanding the algorithm itself. 

I suggest you implement *constraint satisfaction* as it is described in the unit material. A good implementation of a backtracking depth-first search with constraint propagation should be sufficient to get a good grade in the automated tests (roughly 60-70%). There is an example of constraint satisfaction on the eight-queens puzzle, and you could use this as a guide to help you get started.

You could also write a successful agent that uses the other search techniques you have seen in the unit so far: basic search, heuristic search, or local search. You may find these easier to implement, though they may perform less well. 

To get a high grade on this assignment will require a particularly efficient implementation of constraint satisfaction, or something which goes beyond the material we have presented. *This is left unguided and is not factored into the unit workload estimates.*

If you choose to implement more than one algorithm, please feel free to include your code and write about it in part two (readme file), but only the code in this notebook will be used in the automated testing.

## Sample Sudoku Puzzles
To get started, the cell below will load in some sample sudoku puzzles for you so you can see the format. There are sudokus provided of multiple difficulties (easier sudokus typically start with more digits provided). The cell below only loads the easiest, but there is another test cell lower in the notebook which will run your code against all of the provided puzzles.

Each sudoku is a 9x9 NumPy array of integers, where zero represents an empty square. Each difficulty comes with 15 sudokus, so when you load the file, it is stored in a 15x9x9 array.

In [ ]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [ ]:
import numpy as np
import time
import random
import copy


class unsolved_sudoku:
    
    def __init__(self, sudoku_array):
        self.sudoku = sudoku_array
        self.possible_values_for_square = [[[1,2,3,4,5,6,7,8,9] for _ in range(1,9+1)] for _ in range(1, 9+1)]
        self.empty_cells = [[i, j] for i in range(0, 9) for j in range(0, 9) if self.sudoku[i][j] == 0]
        self.possible_values_for_square = self.get_possible_values() 
        self.naked_pairs()  
        
        
    def get_possible_values(self):
        # if value isn't in empty cells 
        for i in range (0,9):
            for j in range (0, 9):
                if [i, j] not in self.empty_cells:
                    self.possible_values_for_square[i][j] = []
                else:
                    self.possible_values_for_square[i][j] = self.possible_values(i, j)
        return(self.possible_values_for_square)
    
    def get_possible_values_for_square(self, row, collumn):
        return (self.possible_values_for_square[row][collumn])
    
    #returns array of possible values for that square (indicated by the row and collumn)
    def possible_values(self, row, collumn):
        possible_values = self.possible_values_for_square[row][collumn]
        ## possible values looking at the row
        row_values = [value for value in self.sudoku[row] if value != 0]
        possible_values = (set(possible_values).difference(row_values))
        ## possible values looking at the collumn
        collumn_values = []
        for i in range(9):
            if self.sudoku[i][collumn] != 0:
                collumn_values.append(self.sudoku[i][collumn])
        possible_values = (set(possible_values).difference(collumn_values)) 
        three_by_three = self.get_three_by_three_from_row(row, collumn)
        three_by_three_values = [value for value in (three_by_three.flatten()) if value != 0]
        possible_values = (set(possible_values).difference(three_by_three_values))
        return(list(possible_values))
    
    def get_three_by_three_from_row(self, row, collumn):
        if row < 3: 
            rows_lower = 0
            rows_upper = 3
        elif row >= 3 and row < 6:
            rows_lower = 3
            rows_upper = 6
        else:
            rows_lower = 6
            rows_upper = 9

        if collumn < 3: 
            collumns_lower = 0
            collumns_upper = 3
        elif collumn >= 3 and collumn < 6:
            collumns_lower = 3
            collumns_upper = 6
        else:
            collumns_lower = 6
            collumns_upper = 9

        return((np.array(self.sudoku)[rows_lower:rows_upper, collumns_lower:collumns_upper]))
    
    
    def get_three_by_three_from_square(self, square, values=False):
        if (square == 0) or (square == 1) or (square == 2):
            rows_lower = 0
            rows_upper = 3
        elif (square == 3) or (square == 4) or (square == 5):
            rows_lower = 3
            rows_upper = 6
        elif (square == 6) or (square == 7) or (square == 8):
            rows_lower = 6
            rows_upper = 9
    
        if (square == 0) or (square == 3) or (square == 6):
            collumns_lower = 0
            collumns_upper = 3
        elif (square == 1) or (square == 4) or (square == 7):
            collumns_lower = 3
            collumns_upper = 6
        elif (square == 2) or (square == 5) or (square == 8):
            collumns_lower = 6
            collumns_upper = 9

        if values == True:
            return((self.sudoku)[rows_lower:rows_upper, collumns_lower:collumns_upper])
        else:
            return([rows_lower, rows_upper, collumns_lower, collumns_upper])
    
    def get_three_by_three_possible_values(self, square):
        if (square == 0) or (square == 1) or (square == 2):
            rows_lower = 0
            rows_upper = 3
        elif (square == 3) or (square == 4) or (square == 5):
            rows_lower = 3
            rows_upper = 6
        elif (square == 6) or (square == 7) or (square == 8):
            rows_lower = 6
            rows_upper = 9
    
        if (square == 0) or (square == 3) or (square == 6):
            collumns_lower = 0
            collumns_upper = 3
        elif (square == 1) or (square == 4) or (square == 7):
            collumns_lower = 3
            collumns_upper = 6
        elif (square == 2) or (square == 5) or (square == 8):
            collumns_lower = 6
            collumns_upper = 9
        
        possible_values = []
        for i in range (rows_lower, rows_upper):
            for j in range (collumns_lower, collumns_upper):
                possible_values.append(self.possible_values_for_square[i][j])

        return(possible_values)
 
    def is_row_solved(self, row):
        if sorted(self.sudoku[row]) == list(range(1, 10)):
            return True
        return False 
    
    def is_collumn_solved(self, collumn):
        collumn_values = []
        for i in range(9):
            collumn_values.append(self.sudoku[i][collumn])
        if sorted(collumn_values) == list(range(1, 10)):
            return True
        return False 
    
    def is_three_by_three_solved(self, square):
        three_by_three = self.get_three_by_three_from_square(square, True)
        if sorted(three_by_three.flatten()) == list(range(1, 10)):
            return True
        return False
    
    def is_invalid(self):
        # print("check invalid")
        dictionary = self.get_possible()
        for key in dictionary:
            if dictionary[key] == []:
                return True
        ## check if each row is invalid 
        for i in range(9):
            if self.is_row_solved(i):
                continue
            else:
                row_values = [value for value in self.sudoku[i] if value != 0]
                if (len(row_values) != len(set(row_values))):
                    return True
                
                
        for i in range(9):
            if self.is_collumn_solved(i):
                continue
            else:
                collumn_values = []
                for j in range(9):
                    if self.sudoku[j][i] != 0:
                        collumn_values.append(self.sudoku[j][i])
                if (len(collumn_values) != len(set(collumn_values))):
                    return True
                
        for i in range(9):
            if self.is_three_by_three_solved(i):
                continue
            else:
                three_by_three = self.get_three_by_three_from_square(i, True)
                three_by_three_values = [value for value in (three_by_three.flatten()) if value != 0]
                if (len(three_by_three_values) != len(set(three_by_three_values))):
                    return True
                
        return False
    
    def set_value(self, row, collumn, number):
        if [row, collumn] not in self.empty_cells:
            raise ValueError(f"{row} {collumn} is not empty so you cannot set a value there")    
        state = copy.deepcopy(self)
        state.empty_cells.remove([row, collumn])
        state.possible_values_for_square[row][collumn].remove(number)
        state.sudoku[row][collumn] = number
        state.possible_values_for_square = state.get_possible_values() 
        state.naked_pairs()
        singleton_squares = state.get_singletons()
        if (len(singleton_squares) != 0):
            row = singleton_squares[0][0]
            collumn = singleton_squares[0][1]
            number = state.possible_values_for_square[row][collumn][0]
            state = state.set_value(singleton_squares[0][0], singleton_squares[0][1], state.possible_values_for_square[row][collumn][0])
        return state
                
    def is_goal(self):
        for i in range(0, 9):
            row_solved = self.is_row_solved(i)
            collumn_solved = self.is_collumn_solved(i)
            square_solved = self.is_three_by_three_solved(i)
            
            if (row_solved == False or collumn_solved == False or square_solved == False):
                return False
        return True
        
 
    def get_possible(self):
        my_dict = {}
        for i in range(len(self.empty_cells)):
            row = self.empty_cells[i][0]
            collumn = self.empty_cells[i][1]
            key = (row, collumn)
            value = self.possible_values_for_square[row][collumn]
            my_dict[key] = value
        return my_dict
    
                
    #returns squares that have no final values and only one possible value
    def get_singletons(self):
        singletons = [] 
        for i in range (len(self.empty_cells)):
            list = []
            row = self.empty_cells[i][0]
            collumn = self.empty_cells[i][1]
            if ((len(self.possible_values_for_square[row][collumn]))==1):
                singletons.append(self.empty_cells[i])
        return(singletons)
    
    def get_final_state(self):
        if self.is_goal():
            return self.sudoku
        else:
            return [[-1 for _ in range(1,9+1)] for _ in range(1, 9+1)]

    def naked_pairs(self):
        ## check if any two squares in the same box, row , collumn have the same two values
        ## if they do that means those two values can only be in those two square in that box/row/collumn so remove it from other possible values

        for i in range(0,9):
            row_possible_values = self.possible_values_for_square[i]
            row_values = {}
            row_naked_pairs = {}
            for j in range(len(row_possible_values)):
                if len(row_possible_values[j]) == 2:
                    row_values[j] = row_possible_values[j]

            for key1 in row_values:
                for key2 in row_values:
                    if key1 != key2:
                        if row_values[key1] == row_values[key2]:
                            row_naked_pairs[(key1,key2)] = row_values[key1]

            for key in row_naked_pairs:
                location1 = key[0]
                location2 = key[1]
                values = row_naked_pairs[key]
                if len(values) > 1:
                    value1 = values[0]
                    value2 = values[1]
                    for index in range (0,9):
                        if ((index != location1) and (index != location2)):
                            if value1 in row_possible_values[index]:
                                if (len(row_possible_values[index]) > 1) and (row_possible_values[index] != [value1, value2]):
                                    row_possible_values[index].remove(value1)
                            if value2 in row_possible_values[index]:
                                if (len(row_possible_values[index]) > 1) and (row_possible_values[index] != [value1, value2]):
                                    row_possible_values[index].remove(value2)
            
            self.possible_values_for_square[i] = row_possible_values
            
            collumn_possible_values = []

            for row in range(9):
                collumn_possible_values.append(self.possible_values_for_square[row][i])
            collumn_values = {}
            collumn_naked_pairs = {}
            for j in range(len(collumn_possible_values)):
                if len(collumn_possible_values[j]) == 2:
                    collumn_values[j] = collumn_possible_values[j]
            for key1 in collumn_values:
                for key2 in collumn_values:
                    if key1 != key2:
                        if collumn_values[key1] == collumn_values[key2]:
                            collumn_naked_pairs[(key1,key2)] = collumn_values[key1]
            
            for key in collumn_naked_pairs:
                pair1 = key[0]
                pair2 = key[1]
                values = collumn_naked_pairs[key]
                if len(values) > 1:
                    value1 = values[0]
                    value2 = values[1]
                    for index in range (0,9):
                        if ((index != pair1) and (index != pair2)):
                            if value1 in collumn_possible_values[index]:
                                if (len(collumn_possible_values[index]) > 1) and (collumn_possible_values[index] != [value1, value2]):
                                    collumn_possible_values[index].remove(value1)
                            if value2 in collumn_possible_values[index]:
                                if (len(collumn_possible_values[index]) > 1) and (collumn_possible_values[index] != [value1, value2]):
                                    collumn_possible_values[index].remove(value2)
            
            for row in range(9):
                self.possible_values_for_square[row][i] = collumn_possible_values[row]
        

            square_possible_values = self.get_three_by_three_possible_values(i)
            square_values = {}
            square_naked_pairs = {}
            for j in range(len(square_possible_values)):
                if len(square_possible_values[j]) == 2:
                    square_values[j] = square_possible_values[j]
            for key1 in square_values:
                for key2 in square_values:
                    if key1 != key2:
                        if square_values[key1] == square_values[key2]:
                            square_naked_pairs[(key1,key2)] = square_values[key1]
            
            for key in square_naked_pairs:
                pair1 = key[0]
                pair2 = key[1]
                values = square_naked_pairs[key]
                if len(values) > 1:
                    value1 = values[0]
                    value2 = values[1]
                    for index in range (0,9):
                        if ((index != pair1) and (index != pair2)):
                            if value1 in square_possible_values[index]:
                                if (len(square_possible_values[index]) > 1) and (square_possible_values[index] != [value1, value2]):
                                    square_possible_values[index].remove(value1)
                            if value2 in square_possible_values[index]:
                                if (len(square_possible_values[index]) > 1) and (square_possible_values[index] != [value1, value2]):
                                    square_possible_values[index].remove(value2)

            rows_lower = self.get_three_by_three_from_square(i)[0]
            rows_upper = self.get_three_by_three_from_square(i)[1]
            collumns_lower = self.get_three_by_three_from_square(i)[2]
            collumns_upper = self.get_three_by_three_from_square(i)[3]
            count = 0
            for row in range (rows_lower, rows_upper):
                for collumn in range (collumns_lower, collumns_upper):
                    self.possible_values_for_square[row][collumn] = square_possible_values[count]
                    count += 1
                    



In [ ]:

"""
Solves a Sudoku puzzle and returns its unique solution.

Input
    sudoku : 9x9 numpy array
        Empty cells are designated by 0.

Output
    9x9 numpy array of integers
        It contains the solution, if there is one. If there is no solution, all array entries should be -1.
"""

def check_validity(sudoku):
    dictionary = sudoku.get_possible()
    for key in dictionary:
        if dictionary[key] == []:
            return False
    return True

def pick_next_square(sudoku):
    """
    Used in depth first search, currently chooses a random 
    square that has more than one possible value
    """
    dictionary = sudoku.get_possible()
    length = 10
    short_key = ()
    for key in dictionary:
        if len(dictionary[key]) < length:
            length = len(dictionary[key])
            short_key = key
    return([short_key[0],short_key[1]])
#     return random.choice(sudoku.empty_cells)


def order_values(sudoku, cell):
    values = sudoku.get_possible_values_for_square(cell[0], cell[1])
    value_dict = {}
    for value in values:
        count = 0
        for row in range (9):
            for collumn in range (9):
                for item in (sudoku.get_possible_values_for_square(row, collumn)):
                    if item == value:
                        count += 1
        value_dict[value] = count
    # print(value_dict)
    lst = []
    value_dict = dict(sorted(value_dict.items(), key=lambda item: item[1]))
    for key in value_dict:
        lst.append(key)
    
    return lst

    """
    Get values for a particular column in the 
    order we should try them in. Currently random.
    """


def depth_first_search(sudoku_board):
        square = pick_next_square(sudoku_board)
        values = order_values(sudoku_board, square)
        for value in values:
            new_state = sudoku_board.set_value(square[0], square[1], value)
            if new_state.is_goal():
                return new_state
            if not new_state.is_invalid():
                deep_state = depth_first_search(new_state)
                if not deep_state.is_invalid() and deep_state.is_goal():
                    return deep_state
        return sudoku_board
            



def sudoku_solver(sudoku):
    UnsolvedSudoku = unsolved_sudoku(sudoku)
    solved_sudoku = depth_first_search(UnsolvedSudoku)
    return(np.array(solved_sudoku.get_final_state()))




All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout. Note that this is a maximum, not a goal. Higher grades require better performance on harder puzzles. As a very rough benchmark, you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms and data structures. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [ ]:
SKIP_TESTS = False

def tests():
    import time
    difficulties = ['very_easy', 'easy', 'medium', 'hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()

Testing very_easy sudokus
This is very_easy sudoku number 0
[[1 0 4 3 8 2 9 5 6]
 [2 0 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 0 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 0 7 2 9 8 5 4 3]
 [8 4 3 0 1 5 2 6 9]]
This is your solution for very_easy sudoku number 0
[[1 7 4 3 8 2 9 5 6]
 [2 9 5 4 6 7 1 3 8]
 [3 8 6 9 5 1 4 7 2]
 [4 6 1 5 2 3 8 9 7]
 [7 3 8 1 4 9 6 2 5]
 [9 5 2 8 7 6 3 1 4]
 [5 2 9 6 3 4 7 8 1]
 [6 1 7 2 9 8 5 4 3]
 [8 4 3 7 1 5 2 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.006718000000006441 seconds to solve.

This is very_easy sudoku number 1
[[0 9 3 1 5 2 6 0 8]
 [8 6 2 7 0 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 0 3 6]
 [5 0 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]
 [6 1 4 2 7 8 9 5 3]
 [7 3 9 6 1 5 8 4 2]
 [2 8 5 3 9 4 7 6 1]]
This is your solution for very_easy sudoku number 1
[[4 9 3 1 5 2 6 7 8]
 [8 6 2 7 4 3 1 9 5]
 [1 5 7 9 8 6 3 2 4]
 [9 7 8 4 2 1 5 3 6]
 [5 2 6 8 3 9 4 1 7]
 [3 4 1 5 6 7 2 8 9]


This is your solution for easy sudoku number 14
[[6 9 7 8 2 3 5 1 4]
 [2 3 5 7 4 1 6 9 8]
 [4 8 1 5 6 9 3 2 7]
 [5 7 2 3 1 4 9 8 6]
 [9 4 8 6 5 2 1 7 3]
 [3 1 6 9 7 8 4 5 2]
 [8 2 3 1 9 6 7 4 5]
 [7 6 9 4 8 5 2 3 1]
 [1 5 4 2 3 7 8 6 9]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.008138999999999896 seconds to solve.

15/15 easy sudokus correct
Testing medium sudokus
This is medium sudoku number 0
[[1 6 0 2 5 3 0 0 8]
 [3 5 2 0 7 8 6 1 4]
 [8 7 9 0 6 4 0 2 3]
 [6 2 3 7 0 0 8 4 9]
 [7 0 5 8 0 0 0 3 6]
 [9 4 8 3 2 5 1 0 0]
 [5 8 6 0 3 2 7 9 1]
 [0 3 7 6 9 1 0 0 5]
 [4 9 0 5 8 7 3 6 0]]
This is your solution for medium sudoku number 0
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took

This is your solution for hard sudoku number 1
[[-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1 -1]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 0.1473450000000014 seconds to solve.

This is hard sudoku number 2
[[0 2 0 0 0 6 9 0 0]
 [0 0 0 0 5 0 0 2 0]
 [6 0 0 3 0 0 0 0 0]
 [9 4 0 0 0 7 0 0 0]
 [0 0 0 4 0 0 7 0 0]
 [0 3 0 2 0 0 0 8 0]
 [0 0 9 0 4 0 0 0 0]
 [3 0 0 9 0 2 0 1 7]
 [0 0 8 0 0 0 0 0 2]]
This is your solution for hard sudoku number 2
[[4 2 5 8 1 6 9 7 3]
 [8 9 3 7 5 4 6 2 1]
 [6 1 7 3 2 9 5 4 8]
 [9 4 1 6 8 7 2 3 5]
 [5 8 2 4 3 1 7 6 9]
 [7 3 6 2 9 5 1 8 4]
 [2 7 9 1 4 8 3 5 6]
 [3 5 4 9 6 2 8 1 7]
 [1 6 8 5 7 3 4 9 2]]
Is your solution correct?
Yes! Correct solution.
This sudoku took 12.111379000000014 seconds to solve.

This is hard sudok

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [ ]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

In [ ]:
# This is a TEST CELL. Do not delete or change.